# Search embedding

Virtual environment
https://www.geeksforgeeks.org/using-jupyter-notebook-in-virtual-environment/

In [1]:
# !pip install -U sentence-transformers

## Load dataset

In [9]:
# Here we store constant varables with path to our datasets
DATA_INDEX_PATH = "../../datasets/"
MODEL_PATH = "../../models/"

SEARCH_DATA_PATH = "../../datasets"

#### load search data

In [11]:
import pandas as pd
# here we load metadata (related to source of dataset)
data_vote = pd.read_json(SEARCH_DATA_PATH+"/search_dataset_vote.json")
data_active = pd.read_json(SEARCH_DATA_PATH+"/search_dataset_active.json")
data_hot = pd.read_json(SEARCH_DATA_PATH+"/search_dataset_hot.json")

# here we load datasets (texts)
search_data_index_hot = pd.read_csv(SEARCH_DATA_INDEX_PATH+"/metadata-for-search-info-hottest-rating.csv")
search_data_index_vote = pd.read_csv(SEARCH_DATA_INDEX_PATH+"/metadata-for-search-info.csv")
search_data_index_active = pd.read_csv(SEARCH_DATA_INDEX_PATH+"/metadata-for-search-info-active-rating.csv")

In [12]:
data_hot['id'], data_hot['text']

(0        0
 1        1
 2        2
 3        3
 4        4
       ... 
 684    685
 685    686
 686    687
 687    688
 688    689
 Name: id, Length: 689, dtype: int64,
 0       Top 10000 Popular Movies Dataset Top 10000 Po...
 1       IMDB Top 1000 Movies Dataset "Release Year, D...
 2       Top 1000 IMDb Movies Dataset Discover the Gre...
 3       Top Rated 10000 Movies Dataset (IMDB) In this...
 4       Top 1000 Highest Grossing Movies Top 1000 Hig...
                              ...                        
 684     Scene Classification Contains ~25K images fro...
 685     Plants Classification 30 Types of Plants Imag...
 686     Flower Classification 14 Types of Flower Imag...
 687     BIRDS 525  SPECIES- IMAGE CLASSIFICATION 525 ...
 688     Brain Tumor Classification (MRI) Classify MRI...
 Name: text, Length: 689, dtype: object)

In [13]:
queries = search_data_index_vote['query name'].unique()

## Models

### Inference batching utils

In [16]:
# this function return batches 
def iterate_batch(dataset,batch_size=128):
    for i in range(0,len(dataset),batch_size):
        yield dataset[i:i+batch_size]

In [17]:
iterate_batch([1,2,3,4,5,6,7,8,9,10],3)

<generator object iterate_batch at 0x7f62712f5f50>

### qween

In [34]:
import torch
from typing import Optional, cast

import numpy as np
import numpy.typing as npt
from chromadb.api.types import EmbeddingFunction, Documents, Embeddings

from torch import Tensor
from transformers import AutoTokenizer, AutoModel

class TransformerEmbeddingFunction_Qween(EmbeddingFunction[Documents]):
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained('Alibaba-NLP/gte-Qwen2-7B-instruct', trust_remote_code=True)
        self.model = AutoModel.from_pretrained('Alibaba-NLP/gte-Qwen2-7B-instruct', trust_remote_code=True)
        
        self.max_length = 8192
        return
    def __call__(self, input: Documents) -> Embeddings:
        
        def last_token_pool(last_hidden_states: Tensor,attention_mask: Tensor) -> Tensor:
            left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
            if left_padding:
                return last_hidden_states[:, -1]
            else:
                sequence_lengths = attention_mask.sum(dim=1) - 1
                batch_size = last_hidden_states.shape[0]
                return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

        batch_dict = self.tokenizer(input, max_length=self.max_length, padding=True, truncation=True, return_tensors='pt')
        outputs = self.model(**batch_dict)
        embeddings = last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

        

        
        return embeddings.tolist()
    
    def preprocess_query(self,text_list):
        task = 'Given a web search query, retrieve relevant passages that answer the query'
        def get_detailed_instruct(task_description: str, query: str) -> str:
            return f'Instruct: {task_description}\nQuery: {query}'

        text_list = [get_detailed_instruct(task, text) for text in text_list]
        return text_list

### Load vector database

In [ ]:
model_embed = TransformerEmbeddingFunction_Qween()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
all_results_hot = []
all_results_vote = []
all_results_active = []

In [ ]:
metrics = {}

In [ ]:
model_name = "qween"

In [ ]:
load_time = 0
search_time = 0

In [ ]:
# client.delete_collection("seach_db_vote")
# client.delete_collection("seach_db_active")
# client.delete_collection("seach_db_hot")

In [ ]:
import chromadb
client = chromadb.Client()

In [ ]:
import chromadb
import time

metrics[model_name] = {"active":{},"vote":{},"hot":{}}

client = chromadb.Client()

collection_vote = client.create_collection("seach_db_vote",get_or_create=False,embedding_function=model_embed)
collection_active = client.create_collection("seach_db_active",get_or_create=False,embedding_function=model_embed)
collection_hot = client.create_collection("seach_db_hot",get_or_create=False,embedding_function=model_embed)
# collection = client.create_collection("test_seach_fb",embedding_function,get_or_create=True)

# load
load_time = time.time()
collection_vote.add(
    documents=list(data_vote['text']),
    ids=[str(i) for i in data_vote['id']]
)
load_time = time.time() - load_time
metrics[model_name]['vote']['load_time'] = load_time

load_time = time.time()
collection_active.add(
    documents=list(data_active['text']),
    ids=[str(i) for i in data_active['id']]
)
load_time = time.time() - load_time
metrics[model_name]['active']['load_time'] = load_time

load_time = time.time()
collection_hot.add(
    documents=list(data_hot['text']),
    ids=[str(i) for i in data_hot['id']]
)
load_time = time.time() - load_time
metrics[model_name]['hot']['load_time'] = load_time

In [ ]:
search_time = time.time()
queries = list(search_data_index_vote['query name'].unique())
queries = model_embed.preprocess_query(queries)
results_vote = collection_vote.query(
    query_texts=queries, # Chroma will embed this for you
    n_results=40 # how many results to return
)
search_time = time.time() - search_time
metrics[model_name]['vote']['search_time'] = search_time

search_time = time.time()
queries = list(search_data_index_hot['query name'].unique())
queries = model_embed.preprocess_query(queries)
results_hot = collection_hot.query(
    query_texts=queries, # Chroma will embed this for you
    n_results=40 # how many results to return
)
search_time = time.time() - search_time
metrics[model_name]['hot']['search_time'] = search_time

search_time = time.time()
queries = list(search_data_index_active['query name'].unique())
queries = model_embed.preprocess_query(queries)
results_active = collection_active.query(
    query_texts=queries, # Chroma will embed this for you
    n_results=40 # how many results to return
)
search_time = time.time() - search_time
metrics[model_name]['active']['search_time'] = search_time

client.delete_collection("seach_db_vote")
client.delete_collection("seach_db_active")
client.delete_collection("seach_db_hot")

## Evaluation


In [ ]:
import sys
sys.path.insert(0, '../../scripts/')

In [ ]:
from evaluation import evaluate_algs

In [ ]:
metrics[model_name]['vote']['metrics'] = evaluate_algs(results_vote,search_data_index_vote)
metrics[model_name]['active']['metrics'] = evaluate_algs(results_active,search_data_index_active)
metrics[model_name]['hot']['metrics'] = evaluate_algs(results_hot,search_data_index_hot)